# API Trading for many exchanges with CCXT

## Installing required Libraries/Packages

Install ccxt with:
- pip install ccxt

(first: conda update anaconda)

## Introduction

In [ ]:
import ccxt

In [ ]:
print(ccxt.exchanges) # all supported exchanges

__Binance.com (Spot)__

In [ ]:
binance = ccxt.binance()
binance

In [ ]:
binance.

In [ ]:
binance.fetchTicker(symbol = 'BTC/USDT') # get current market data

__Binance.US (Spot)__

In [ ]:
binanceus = ccxt.binanceus()
binanceus

In [ ]:
binanceus.fetchTicker(symbol = 'BTC/USD') # get current market data

__Binance Futures (USD-margined)__

In [ ]:
binanceusdm = ccxt.binanceusdm()
binanceusdm

In [ ]:
binanceusdm.fetchTicker(symbol = 'BTC/USDT') # get current market data

__Binance Futures (Coin-margined)__

In [ ]:
binancecoinm = ccxt.binancecoinm()
binancecoinm

In [ ]:
binancecoinm.fetchTicker(symbol = 'BTC/USD') # get current market data

__coinbasepro__

In [ ]:
coinbasepro = ccxt.coinbasepro()
coinbasepro

In [ ]:
coinbasepro.fetchTicker(symbol = 'BTC/USDT') # get current market data

__FTX.com__ # removed

In [ ]:
ftx = ccxt.ftx() # removed
ftx

In [ ]:
ftx.fetchTicker(symbol = 'BTC/USDT') # get current market data

__FTX.US__ # removed

In [ ]:
ftxus = ccxt.ftxus() # removed
ftxus

In [ ]:
ftxus.fetchTicker(symbol = 'BTC/USDT') # get current market data

__bybit__

In [ ]:
bybit = ccxt.bybit()
bybit

In [ ]:
bybit.fetchTicker(symbol = 'BTC/USDT') # get current market data

__kraken__

In [ ]:
kraken = ccxt.kraken()
kraken

In [ ]:
kraken.fetchTicker(symbol = 'BTC/USDT') # get current market data

## General Exchange Information

In [ ]:
import ccxt
import pandas as pd

In [ ]:
binance = ccxt.binance()
binance

In [ ]:
binance.id

In [ ]:
binance.name

In [ ]:
binance.has # most important/common methods among the exchanges ("Unified API")

In [ ]:
binance.timeframes # available timeframes / frequencies

In [ ]:
binance.loadMarkets() # available currency pairs

In [ ]:
binance.loadMarkets()["BTC/USDT"] # get more info on a specific pair

In [ ]:
pd.DataFrame(binance.loadMarkets()).T

In [ ]:
binance.symbols # currency pairs (just symbol)

In [ ]:
len(binance.symbols)

In [ ]:
binance.currencies # info on coins

## Public API

Public -> we don´t need an account/api-keys to pull market data

In [ ]:
import ccxt
import pandas as pd

In [ ]:
binance = ccxt.binance()
binance

In [ ]:
binance.has

In [ ]:
binance.fetchTicker(symbol = 'BTC/USDT') # get current market data for one symbol

In [ ]:
binance.fetchTickers(symbols = ['BTC/USDT', "ETH/USDT"])  # get current market data for many symbols

In [ ]:
binance.fetchOHLCV(symbol = "BTC/USDT", timeframe = "1d", limit = 1000) # OHLCV for one symbol

In [ ]:
# OHLCV for one symbol (index)
# index price == bucket of prices from the major exchanges
binance.fetchIndexOHLCV(symbol = "BTC/USDT", timeframe = "1d", limit = 1000)

In [ ]:
binance.fetchOrderBook(symbol = "BTC/USDT") # current Order Book for one symbol

In [ ]:
binance.fetchTrades(symbol = "BTC/USDT") # most recent Trades for one symbol

## Loading Historical Data (Part 1)

In [ ]:
import ccxt
import pandas as pd

In [ ]:
binance = ccxt.binance()
binance

In [ ]:
binance.timeframes

In [ ]:
# last 1000 bars (max. 1000)
data = binance.fetchOHLCV(symbol = "BTC/USDT", timeframe = "1d", limit = 1000) 
data

In [ ]:
len(data)

In [ ]:
start = "2020-05-15 00:00:00" # UTC time

In [ ]:
unix = binance.parse8601(start)
unix

In [ ]:
binance.iso8601(unix)

In [ ]:
# since ... but max 1000 bars 
data = binance.fetchOHLCV(symbol = "BTC/USDT", timeframe = "1d", since = unix, limit = 1000) 
data

In [ ]:
len(data)

In [ ]:
def get_history(symbol, interval, start = None, limit = 1000):
    
    if start:
        start = binance.parse8601(start)
    
    data = binance.fetchOHLCV(symbol = symbol, timeframe = interval, since = start, limit = limit)
    
    df = pd.DataFrame(data)
    df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
    df.Date = pd.to_datetime(df.Date, unit = "ms")
    df.set_index("Date", inplace = True)

    return df

In [ ]:
df = get_history(symbol = "BTC/USDT", interval = "1h", limit = 1000) # last x bars
df

In [ ]:
df.info()

In [ ]:
start = "2021-11-15 09:00:00"

In [ ]:
# since ... but max 1000 bars
df = get_history(symbol = "BTC/USDT", interval = "1h", start = start, limit = 1000) 
df

In [ ]:
start = "2020-11-15 09:00:00"

In [ ]:
# since ... but max 1000 bars
df = get_history(symbol = "BTC/USDT", interval = "1h", start = start, limit = 1000)
df

## Loading Historical Data (Part 2)

Solution: Loading data in batches of 1,000 bars each (until we reach the current bar)

In [ ]:
import time

In [ ]:
def get_history(symbol, interval, start = None, limit = 1000):
    
    if start:
        start = binance.parse8601(start)
    
    data = binance.fetchOHLCV(symbol = symbol, timeframe = interval, since = start, limit = limit)
    last_bar_actual = data[-1][0] # timestamp of last loaded bar
    
    # timestamp of current bar 
    last_bar_target = binance.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]
    
    # as long as we don´t have all bars (most recent): let´s pull the next 1000 bars
    while last_bar_target != last_bar_actual: 
        
        time.sleep(0.1)
        data_add = binance.fetchOHLCV(symbol = symbol, timeframe = interval,
                                      since = last_bar_actual, limit = limit)
        data += data_add[1:]
        last_bar_actual = data[-1][0]
        last_bar_target = binance.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]      
    
    df = pd.DataFrame(data)
    df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
    df.Date = pd.to_datetime(df.Date, unit = "ms")
    df.set_index("Date", inplace = True)

    return df

In [ ]:
start = "2020-11-15 09:00:00"

In [ ]:
# since ... but max 1000 bars
df = get_history(symbol = "BTC/USDT", interval = "1h", start = start, limit = 1000)
df

In [ ]:
df.info()

In [ ]:
df.index.is_unique

## Streaming real-time Data (Part 1)

Problem: Websocket API is not available in free version

-> Not a Problem if we don´t need (ultra) High-Frequency Data (many updates per second with very low latency)

Solution: frequently pull last/most current bar via the Rest API

-> We need to be careful with API Rate limits (maximum number of calls) 

In [ ]:
import ccxt
import pandas as pd
import time

In [ ]:
binance = ccxt.binance()
binance

In [ ]:
for i in range(10):
    data = binance.fetchOHLCV(symbol = "BTC/USDT", timeframe = "1m", limit = 1)
    print(data)
    time.sleep(1)

In [ ]:
def start_kline_stream(callback, symbol, interval):
    
    for i in range(10):
        msg = binance.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 1)
        
        if len(msg) == 0:
            print("No data received")
        else:
            callback(msg)
    
        time.sleep(1)

In [ ]:
def stream_candles(msg):
    # defines how to process the msg
    
    print(msg)

In [ ]:
start_kline_stream(callback = stream_candles, symbol = "BTC/USDT", interval = "1m")

## Introduction to Multithreaded Programming

Goal: Running multiple tasks/sub-programs in parallel (simultaniously)

In [ ]:
import time

In [ ]:
def print_integers(start, stop):
    for i in range(start, stop):
        print(i)
        time.sleep(1)

In [ ]:
print_integers(1, 10) # task 1

In [ ]:
# task 2
a = 5 + 8
print("a is {}.".format(a))

In [ ]:
from threading import Thread

In [ ]:
thread = Thread(target = print_integers, args = (1, 10))
thread

In [ ]:
thread.start() # task 1

In [ ]:
# task 2
a = 5 + 8
print("a is {}.".format(a))

## Streaming real-time Data (Part 2)

In [ ]:
import ccxt
import pandas as pd
import time
from threading import Thread

In [ ]:
binance = ccxt.binance()
binance

In [ ]:
def start_kline_stream(callback, symbol, interval):
    
    global running
    running = True
    
    while running == True:
        msg = binance.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 1)
        
        if len(msg) == 0:
            print("No data received")
        else:
            callback(msg)
    
        time.sleep(1)

In [ ]:
def stream_candles(msg):
    # defines how to process the msg
    
    print(msg)

In [ ]:
def stop_stream():
    global running
    running = False

In [ ]:
thread = Thread(target = start_kline_stream, args = (stream_candles, "BTC/USDT", "1m"))
thread

In [ ]:
thread.start()

In [ ]:
stop_stream()

## Streaming real-time Data (Part 3)

In [ ]:
df = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume"])
df

In [ ]:
def stream_candles(msg):
    # defines how to process the msg
    
    # extract data form msg
    start_time = pd.to_datetime(msg[0][0], unit = "ms")
    first = msg[0][1]
    high = msg[0][2]
    low = msg[0][3]
    close  = msg[0][4]
    volume = msg[0][5]
    
    # feed df
    df.loc[start_time] = [first, high, low, close, volume]
    
    # print something
    print(".", end = "", flush = True)

In [ ]:
thread = Thread(target = start_kline_stream, args = (stream_candles, "BTC/USDT", "1m"))
thread

In [ ]:
thread.start()

In [ ]:
stop_stream()

In [ ]:
df

## Get Historical Data and Stream live Data

In [ ]:
import ccxt
import pandas as pd
import time
from threading import Thread

In [ ]:
exchange = ccxt.binance()
exchange

In [ ]:
class CCXTTrader(): # without execution code
    
    def __init__(self, symbol, bar_length):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.get_available_intervals()
    
    def get_available_intervals(self):
        
        l = []
        for key, value in exchange.timeframes.items():
            l.append(key)
        self.available_intervals = l
    
    def start_trading(self, start = None, hist_bars = None):
        
        if not hist_bars:
            hist_bars = 1000
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 start = start, limit = hist_bars)
            thread = Thread(target = self.start_kline_stream, args = (self.stream_candles, self.symbol, self.bar_length))
            thread.start()
            
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, start, limit):
        
        if start:
            start = exchange.parse8601(start)
    
        data = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = start, limit = limit)
        last_bar_actual = data[-1][0]
    
        # timestamp of current bar
        last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]
    
        # as long as we don´t have all bars (most recent): let´s pull the next 1000 bars
        while last_bar_target != last_bar_actual:
        
            time.sleep(0.1)
            data_add = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = last_bar_actual, limit = limit)
            data += data_add[1:]
            last_bar_actual = data[-1][0]
            last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]      
    
        df = pd.DataFrame(data)
        df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
        df.Date = pd.to_datetime(df.Date, unit = "ms")
        df.set_index("Date", inplace = True)
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        self.last_bar = df.index[-1]

        self.data = df
        
    def stream_candles(self, msg):
        # defines how to process the msg
    
        # extract data form msg
        start_time = pd.to_datetime(msg[-1][0], unit = "ms")
        first = msg[-1][1]
        high = msg[-1][2]
        low = msg[-1][3]
        close  = msg[-1][4]
        volume = msg[-1][5]
        
        # check if a bar is complete
        if start_time == self.last_bar: # not complete
            complete = False
        else: # complete
            complete = True
            if len(msg) == 2:
                self.data.loc[self.last_bar] = [msg[0][1], msg[0][2], msg[0][3], msg[0][4], msg[0][5], complete]
            else:
                self.data.loc[self.last_bar, "Complete"] = complete
            self.last_bar = start_time
        
        # print something
        print(".", end = "", flush = True)
        
        # feed df with latest bar
        self.data.loc[start_time] = [first, high, low, close, volume, False]
        
        # if a bar is complete, define strategy and trade
        if complete == True:
            print("\n", "Define Strategy and execute Trades") # just a placeholder
            #self.define_strategy()
            #self.execute_trades()
    
    def start_kline_stream(self, callback, symbol, interval):
    
        self.running = True
    
        while self.running == True:
            msg = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)
        
            if len(msg) == 0:
                print("No data received")
            else:
                callback(msg)
    
            time.sleep(1)
    
    def stop_stream(self):
        self.running = False        

In [ ]:
trader = CCXTTrader(symbol = "BTC/USDT", bar_length = "1m")
trader

In [ ]:
trader.start_trading(start = None, hist_bars = 10)

In [ ]:
trader.stop_stream()

In [ ]:
trader.data

## Private API

In [ ]:
import ccxt
import pandas as pd

In [ ]:
binance = ccxt.binance() # in case you have a binance.us account: ccxt.binanceus()
binance

__Copy/Paste your (real account) login credentials here:__

In [ ]:
api_key = "insert here"
secret_key = "insert here"

In [ ]:
binance.apiKey = api_key
binance.secret = secret_key

In [ ]:
binance.fetchBalance()["info"]["balances"] # get asset balances

In [ ]:
trades = binance.fetchMyTrades(symbol = "LINK/BUSD") # recent trades
trades

In [ ]:
order_id = trades[0]["order"]
order_id

In [ ]:
binance.fetchOrder(id = order_id, symbol = "LINK/BUSD") # specific order (id)

In [ ]:
binance.fetchOrders(symbol = "LINK/BUSD") # recent orders

In [ ]:
binance.fetchOpenOrders(symbol = "LINK/BUSD") # open orders

In [ ]:
binance.fetchClosedOrders(symbol = "LINK/BUSD") # closed orders

## Use the Testnets

In [ ]:
import ccxt
import pandas as pd

In [ ]:
binance_t = ccxt.binance()
binance_t

In [ ]:
binance_t.set_sandbox_mode(True) # Binance Spot Testnet

__Copy/Paste your (spot testnet account) login credentials here:__

In [ ]:
api_key = "insert here"
secret_key = "insert here"

In [ ]:
binance_t.apiKey = api_key
binance_t.secret = secret_key

In [ ]:
binance_t.fetchBalance()["info"]["balances"]

In [ ]:
ftx = ccxt.ftx() # removed
ftx

In [ ]:
ftx.set_sandbox_mode(True)

In [ ]:
bybit = ccxt.bybit()
bybit

In [ ]:
bybit.set_sandbox_mode(True)

## Creating Orders (in the Spot Testnet)

In [ ]:
binance_t.fetchBalance()["info"]["balances"]

In [ ]:
binance_t.has["createMarketOrder"]

In [ ]:
# buy 0.01 BTC
order = binance_t.createMarketOrder(symbol = "BTC/USDT", side = "BUY", amount = 0.01)
order

In [ ]:
binance_t.fetchBalance()["info"]["balances"]

In [ ]:
# Sell 0.01 BTC
order = binance_t.createMarketOrder(symbol = "BTC/USDT", side = "SELL", amount = 0.01)
order

In [ ]:
# Buy X BTC for 5,000 USDT
order = binance_t.createMarketOrder(symbol = "BTC/USDT", side = "BUY", amount = 1, price = 5000)
order

In [ ]:
amount = float(order["info"]['executedQty'])
amount

In [ ]:
order = binance_t.createMarketOrder(symbol = "BTC/USDT", side = "SELL", amount = amount)
order

In [ ]:
binance_t.fetchBalance()["info"]["balances"]

## Futures Testnet

In [ ]:
import ccxt
import pandas as pd

In [ ]:
# usd-margined Futures
binanceusdm = ccxt.binanceusdm()
binanceusdm

__Copy/Paste your (futures testnet account) login credentials here:__

In [ ]:
api_key = "insert here"
secret_key = "insert here"

In [ ]:
binanceusdm.apiKey = api_key
binanceusdm.secret = secret_key

In [ ]:
binanceusdm.set_sandbox_mode(True)

In [ ]:
binanceusdm.has

In [ ]:
binanceusdm.fetchBalance()["info"]["assets"]

In [ ]:
# Go Short 0.001 BTC
order = binanceusdm.createMarketOrder(symbol = "BTC/USDT", side = "SELL", amount = 0.001)
order

In [ ]:
binanceusdm.fetchPositions(symbols = ["BTC/USDT"]) # get open positions

In [ ]:
binanceusdm.fetchMyTrades(symbol = "BTC/USDT")[-1] # get recent trades

In [ ]:
# Close Short Position
order = binanceusdm.createMarketOrder(symbol = "BTC/USDT", side = "BUY", amount = 0.001)
order

In [ ]:
binanceusdm.fetchPositions(symbols = ["BTC/USDT"]) # get open positions

In [ ]:
binanceusdm.fetchMyTrades(symbol = "BTC/USDT")[-1] # get recent trades

In [ ]:
binanceusdm.fetchPositions(symbols = ["BTC/USDT"])[0]["info"]["leverage"] # check leverage

In [ ]:
binanceusdm.set_leverage(leverage = 15, symbol = "BTC/USDT") # set leverage

In [ ]:
binanceusdm.fetchPositions(symbols = ["BTC/USDT"])[0]["info"]["marginType"] # check margin mode

In [ ]:
binanceusdm.set_margin_mode(marginMode = "isolated", symbol = "BTC/USDT") # set margin mode

## Algorithmic Spot Trading with Binance and CCXT

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with the Binance Spot Testnet and NOT with a Live Trading Account._

In [ ]:
import pandas as pd
import numpy as np
import time
import ccxt
from threading import Thread

In [ ]:
class CCXTSpotTrader(): # based on Long-Short Trader (Contrarian Strategy)
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh,
                 units, position = 0, sandbox = True):
        
        exchange.set_sandbox_mode(sandbox) # NEW!
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.get_available_intervals()
        self.units = units
        self.position = position
        self.trades = 0 
        self.trade_values = []
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def get_available_intervals(self):
        
        l = []
        for key, value in exchange.timeframes.items():
            l.append(key)
        self.available_intervals = l
    
    def start_trading(self, start = None, hist_bars = None):
        
        if not hist_bars:
            hist_bars = 1000
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 start = start, limit = hist_bars)
            thread = Thread(target = self.start_kline_stream, args = (self.stream_candles, self.symbol, self.bar_length))
            thread.start()
            
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, start, limit):
        
        if start:
            start = exchange.parse8601(start)
    
        data = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = start, limit = limit)
        last_bar_actual = data[-1][0]
    
        # timestamp of current bar
        last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]
    
        # as long as we don´t have all bars (most recent): let´s pull the next 1000 bars
        while last_bar_target != last_bar_actual:
        
            time.sleep(0.1)
            data_add = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = last_bar_actual, limit = limit)
            data += data_add[1:]
            last_bar_actual = data[-1][0]
            last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]      
    
        df = pd.DataFrame(data)
        df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
        df.Date = pd.to_datetime(df.Date, unit = "ms")
        df.set_index("Date", inplace = True)
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        self.last_bar = df.index[-1]

        self.data = df
        
    def stream_candles(self, msg):
        # defines how to process the msg
    
        # extract data form msg
        start_time = pd.to_datetime(msg[-1][0], unit = "ms")
        first = msg[-1][1]
        high = msg[-1][2]
        low = msg[-1][3]
        close  = msg[-1][4]
        volume = msg[-1][5]
        
        # check if a bar is complete
        if start_time == self.last_bar:
            complete = False
        else:
            complete = True
            if len(msg) == 2:
                self.data.loc[self.last_bar] = [msg[0][1], msg[0][2], msg[0][3], msg[0][4], msg[0][5], complete]
            else:
                self.data.loc[self.last_bar, "Complete"] = complete
            self.last_bar = start_time
        
        # print something
        print(".", end = "", flush = True)
        
        # feed df with latest bar
        self.data.loc[start_time] = [first, high, low, close, volume, False]
        
        # if a bar is complete, define strategy and trade
        if complete == True:
            self.define_strategy()
            self.execute_trades()
    
    def start_kline_stream(self, callback, symbol, interval):
    
        self.running = True
    
        while self.running == True:
            msg = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)
        
            if len(msg) == 0:
                print("No data received")
            else:
                callback(msg)
    
            time.sleep(1)
    
    def stop_stream(self):
        self.running = False     
         
    def define_strategy(self):
        
        df = self.data.loc[self.data.Complete == True].copy() # Adj!
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns <= self.return_thresh[0]
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        cond3 = df.returns >= self.return_thresh[1]
        
        df["position"] = 0
        df.loc[cond1 & cond2, "position"] = 1
        df.loc[cond3 & cond2, "position"] = -1
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "BUY", amount = self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "BUY", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL")
                time.sleep(0.1)
                order = exchange.createMarketOrder(symbol = self.symbol, side = "BUY", amount = self.units)
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "SELL", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == -1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "BUY", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            self.position = 0
        if self.prepared_data["position"].iloc[-1] == -1: # if position is short -> go/stay short
            if self.position == 0:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "SELL", amount = self.units)
                self.report_trade(order, "GOING SHORT") 
            elif self.position == 1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "SELL", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL")
                time.sleep(0.1)
                order = exchange.createMarketOrder(symbol = self.symbol, side = "SELL", amount = self.units)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
            
    def report_trade(self, order, going): 
        
        # extract data from order object (Adj!!!)
        side = order["side"].upper()
        time = pd.to_datetime(order["timestamp"], unit = "ms")
        base_units = float(order["filled"])
        quote_units = float(order["cost"])
        price = float(order["average"])
        fee_asset = order["fees"][0]["currency"]
        fee_amount: order["fees"][0]["cost"]
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            self.cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            self.cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")
        

In [ ]:
exchange = ccxt.binance()
exchange

__Copy/Paste your (Spot testnet account) login credentials here:__

In [ ]:
api_key = "insert here"
secret_key = "insert here"

In [ ]:
exchange.apiKey = api_key
exchange.secret = secret_key

In [ ]:
symbol = "BTC/USDT"
bar_length = "1m"
return_thresh = [-0.0001, 0.0001]
volume_thresh = [-3, 3]
units = 0.01
position = 0

In [ ]:
trader = CCXTSpotTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                        volume_thresh = volume_thresh, units = units, position = 0, sandbox = True)

In [ ]:
exchange.fetchBalance()["info"]["balances"] # get asset balances

In [ ]:
trader.start_trading(start = None, hist_bars = 10)

In [ ]:
trader.stop_stream()

In [ ]:
trader.prepared_data

In [ ]:
exchange.fetchBalance()["info"]["balances"] # get asset balances

## Excursus: Generalization / Covering all cases

- CCXTSpotTrader works with the Binance Spot API
- The more exchanges we include, the more __(strange) behaviors__ we have to handle/take into account
- this makes the (generalized) Trader Class __more complex__ and harder to read/understand

__Example: Pulling the two most recent bars ("live stream")__

__Desired outcome__: We receive the __two most recent bars__

__What can go wrong?__ (all potential cases):

- __no__ bar/empty output (already covered in the class)

- only __one__ bar (already covered in the class)

- __error__ (to be fixed with error handling techniques -> later)

- not the two most recent bars (typically second and third most recent bar -> __most recent bar missing__)

In [ ]:
def stream_candles(self, msg):
        # defines how to process the msg
    
        # extract data form msg
        start_time = pd.to_datetime(msg[-1][0], unit = "ms")
        first = msg[-1][1]
        high = msg[-1][2]
        low = msg[-1][3]
        close  = msg[-1][4]
        volume = msg[-1][5]
        
        # if most recent bar is suddenly missing
        if start_time < self.last_bar:
            pass # do nothing and pull the next msg
            
        else:
            if start_time == self.last_bar:
                complete = False
            elif start_time > self.last_bar:
                complete = True
                if len(msg) == 2:
                    self.data.loc[self.last_bar] = [msg[0][1], msg[0][2], msg[0][3], msg[0][4], msg[0][5], complete]
                else:
                    self.data.loc[self.last_bar, "Complete"] = complete
                self.last_bar = start_time
            
            # print something
            print(".", end = "", flush = True)
        
            # feed df with latest bar
            self.data.loc[start_time] = [first, high, low, close, volume, False]
        
            # if a bar is complete, define strategy and trade
            if complete == True:
                self.define_strategy()
                self.execute_trades()

-> required for the Binance Futures API!

## Algorithmic Futures Trading with Binance and CCXT

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with the Binance Futures Testnet and NOT with a Live Trading Account._

In [ ]:
import pandas as pd
import numpy as np
import time
import ccxt
from threading import Thread

In [ ]:
class CCXTFuturesTrader(): # Based on FuturesTrader (Contrarian)
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh,
                 units, position = 0, leverage = 5, sandbox = True):
        
        exchange.set_sandbox_mode(sandbox)
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.get_available_intervals()
        self.units = units
        self.position = position
        self.leverage = leverage
        self.cum_profits = 0
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def get_available_intervals(self):
        
        l = []
        for key, value in exchange.timeframes.items():
            l.append(key)
        self.available_intervals = l
    
    def start_trading(self, start = None, hist_bars = None):
        
        if not hist_bars:
            hist_bars = 1000
        
        exchange.set_leverage(leverage = self.leverage, symbol = self.symbol)
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 start = start, limit = hist_bars)
            thread = Thread(target = self.start_kline_stream, args = (self.stream_candles, self.symbol, self.bar_length))
            thread.start()
            
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, start, limit):
        
        if start:
            start = exchange.parse8601(start)
    
        data = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = start, limit = limit)
        last_bar_actual = data[-1][0]
    
        # timestamp of current bar
        last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]
    
        # as long as we don´t have all bars (most recent): let´s pull the next 1000 bars
        while last_bar_target != last_bar_actual:
        
            time.sleep(0.1)
            data_add = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = last_bar_actual, limit = limit)
            data += data_add[1:]
            last_bar_actual = data[-1][0]
            last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]      
    
        df = pd.DataFrame(data)
        df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
        df.Date = pd.to_datetime(df.Date, unit = "ms")
        df.set_index("Date", inplace = True)
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        self.last_bar = df.index[-1]

        self.data = df
        
    def stream_candles(self, msg):
        # defines how to process the msg
    
        # extract data form msg
        start_time = pd.to_datetime(msg[-1][0], unit = "ms")
        first = msg[-1][1]
        high = msg[-1][2]
        low = msg[-1][3]
        close  = msg[-1][4]
        volume = msg[-1][5]
        
        if start_time < self.last_bar:
            pass
            
        else:
            if start_time == self.last_bar:
                complete = False
            elif start_time > self.last_bar:
                complete = True
                if len(msg) == 2:
                    self.data.loc[self.last_bar] = [msg[0][1], msg[0][2], msg[0][3], msg[0][4], msg[0][5], complete]
                else:
                    self.data.loc[self.last_bar, "Complete"] = complete
                self.last_bar = start_time
            
            # print something
            print(".", end = "", flush = True)
        
            # feed df with latest bar
            self.data.loc[start_time] = [first, high, low, close, volume, False]
        
            # if a bar is complete, define strategy and trade
            if complete == True:
                self.define_strategy()
                self.execute_trades()
    
    def start_kline_stream(self, callback, symbol, interval):
    
        self.running = True
    
        while self.running == True:
            msg = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)
        
            if len(msg) == 0:
                print("No data received")
            else:
                callback(msg)
    
            time.sleep(1)
    
    def stop_stream(self):
        self.running = False     
         
    def define_strategy(self):
        
        df = self.data.loc[self.data.Complete == True].copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns <= self.return_thresh[0]
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        cond3 = df.returns >= self.return_thresh[1]
        
        df["position"] = 0
        df.loc[cond1 & cond2, "position"] = 1
        df.loc[cond3 & cond2, "position"] = -1
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "BUY", amount = self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "BUY", amount = 2 * self.units)
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "SELL", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == -1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "BUY", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
        if self.prepared_data["position"].iloc[-1] == -1: # if position is short -> go/stay short
            if self.position == 0:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "SELL", amount = self.units)
                self.report_trade(order, "GOING SHORT") 
            elif self.position == 1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "SELL", amount = 2 * self.units)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
    
    def report_trade(self, order, going): # Adj!
        
        time.sleep(0.1)
        order_time = int(order["info"]["updateTime"])
        trades = exchange.fetchMyTrades(symbol = self.symbol, since = order_time)
        order_time = pd.to_datetime(order_time, unit = "ms")
        
        # extract data from trade object
        df = pd.json_normalize(trades)
        columns = ["amount", "cost", "info.commission","info.realizedPnl"]
        for column in columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        base_units = round(df["amount"].sum(), 5)
        quote_units = round(df["cost"].sum(), 5)
        commission = -round(df["info.commission"].sum(), 5)
        real_profit = round(df["info.realizedPnl"].sum(), 5)
        price = round(quote_units / base_units, 5)
        
        # calculate cumulative trading profits
        self.cum_profits += round((commission + real_profit), 5)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(order_time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(order_time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(order_time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")

In [ ]:
exchange = ccxt.binanceusdm() 
exchange

__Copy/Paste your (Futures testnet account) login credentials here:__

In [ ]:
api_key = "insert here"
secret_key = "insert here"

In [ ]:
exchange.apiKey = api_key
exchange.secret = secret_key

In [ ]:
symbol = "BTC/USDT"
bar_length = "1m"
return_thresh = [-0.0001, 0.0001]
volume_thresh = [-3, 3]
units = 0.01
position = 0
leverage = 5
sandbox = True

In [ ]:
trader = CCXTFuturesTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                           volume_thresh = volume_thresh, units = units, position = position,
                           leverage = leverage, sandbox = sandbox)

In [ ]:
exchange.fetchBalance()["info"]["assets"] # get asset balances

In [ ]:
trader.start_trading(start = None, hist_bars = 10)

In [ ]:
trader.stop_stream()

In [ ]:
trader.prepared_data

In [ ]:
exchange.fetchBalance()["info"]["assets"] # get asset balances